<a href="https://colab.research.google.com/github/sharonwu827/Deep-Learning/blob/master/AES_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#load dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# import package
!pip install transformers
!pip install bert-tensorflow 
!pip install pyenchant
!pip install -U pip setuptools wheel
!pip install -U spacy
!sudo apt-get install libenchant1c2a
!python -m spacy download en_core_web_sm

import warnings
warnings.filterwarnings('ignore')

import numpy as np 
from numpy import asarray
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer

import re
import time
import torch
import transformers as ppb

from nltk.corpus import stopwords
import torch
import torch.nn as nn
import transformers
from transformers import BertTokenizer, BertModel, AutoModel, BertTokenizerFast,BertForSequenceClassification

# Keras functional API
from keras.models import Sequential, Model, load_model, model_from_config
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Flatten, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K

import os
import csv
import copy

import tensorflow as tf
import tensorflow_hub as hub

import enchant #  Enchant spellchecking library
import spacy
import re

import urllib.request
from bs4 import BeautifulSoup
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libenchant1c2a is already the newest version (1.6.0-11.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
2021-07-16 04:44:00.496293: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.6 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1')
dev = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/valid_set.tsv",sep='\t', encoding='ISO-8859-1')
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/test_set.tsv",sep='\t', encoding='ISO-8859-1')

In [5]:
prompt = pd.DataFrame({"essay_set":[1,2,3,4,5,6,7,8],
                       "prompt":["More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.",
                                "Write a persuasive essay to a newspaper reflecting your vies on censorship in libraries. Do you believe that certain materials, such as books, music, movies, magazines, etc., should be removed from the shelves if they are found offensive? Support your position with convincing arguments from your own experience, observations, and/or reading.",
                                "Write a response that explains how the features of the setting affect the cyclist. In your response, include examples from the essay that support your conclusion.",
                                "Write a response that explains why the author concludes the story with this paragraph. In your response, include details and examples from the story that support your ideas.",
                                "Describe the mood created by the author in the memoir. Support your answer with relevant and specific information from the memoir.",
                                "Based on the excerpt, describe the obstacles the builders of the Empire State Building faced in attempting to allow dirigibles to dock there. Support your answer with relevant and specific information from the excerpt.",
                                "Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining.Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience.",
                                "We all understand the benefits of laughter. For example, someone once said, “Laughter is the shortest distance between two people.” Many other people believe that laughter is an important part of any relationship. Tell a true story in which laughter was one element or part."]})

## Data processing

In [6]:
def preprocess(df):
  df['normalized_score'] = df['domain1_score'] / df.groupby('essay_set')['domain1_score'].transform('max')

preprocess(train)

In [7]:
def clean_anonymization(essay):
  '''
  function to remove the anoymaization
  '''
  res=[]
  # Lowercase
  essay = essay.lower()
  # Replace everything not a letter or apostrophe with a space
  essay = re.sub('[^a-zA-Z\']', ' ', essay)
  # Remove single letter words
  essay = ' '.join( [w for w in essay.split() if len(w)>1] )
  for i in essay.split():
    if i.startswith("@"):
      continue
    else:
      res.append(i)
  return ' '.join(res)

train['essay']=train['essay'].apply(lambda x:clean_anonymization(x))
# also remove from dev and test
dev['essay']=dev['essay'].apply(lambda x:clean_anonymization(x))
test['essay']=test['essay'].apply(lambda x:clean_anonymization(x))

In [8]:
#-------------- change here------------------#
X = train[:1000][['essay']]
y = train[:1000]['normalized_score']

## Load Model

In [9]:
# Load pre-trained model (weights)
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def lstm_model():
    """
    Define the model
    """
    model = Sequential()

    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1,768], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    model.summary()

    return model

## Semantic Score

In [11]:
total_loss = []
results = []
prediction_list = []

cv = KFold(n_splits=5, shuffle=True)
cv_data = cv.split(X)

fold_cnt =1
cuda = torch.device('cuda')

with torch.cuda.device(cuda):
  for traincv, testcv in cv_data:
    torch.cuda.empty_cache()
    print("Fold {}".format(fold_cnt))
    
    # get the train and test from the dataset.
    X_train, X_test, y_train, y_test = X.iloc[traincv], X.iloc[testcv], y.iloc[traincv], y.iloc[testcv]
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    tokenized_train = train_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length=200)))
    tokenized_test = test_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))
    
    # train dataset
    max_len = 0
    for i in tokenized_train.values:
      if len(i) > max_len:
        max_len = len(i)
    padded_train = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])
    
    attention_mask_train = np.where(padded_train != 0, 1, 0)
    train_input_ids = torch.tensor(padded_train)
    train_attention_mask = torch.tensor(attention_mask_train)
    
    with torch.no_grad():
      outputs_train  = model(train_input_ids, attention_mask=train_attention_mask)
      last_hidden_states_train = outputs_train[0]  # get the last hidden state
    train_features = last_hidden_states_train[:,0,:].numpy()
    
    ## test dataset
    max_len = 0
    for i in tokenized_test.values:
      if len(i) > max_len:
        max_len = len(i)
    padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])
    
    attention_mask_test = np.where(padded_test != 0, 1, 0)
    test_input_ids = torch.tensor(padded_test)  
    test_attention_mask = torch.tensor(attention_mask_test)

    with torch.no_grad():
      outputs_test  = model(test_input_ids, attention_mask=test_attention_mask)
      last_hidden_states_test = outputs_test[0]  # get the last hidden state
    test_features = last_hidden_states_test[:,0,:].numpy()
    train_x,train_y = train_features.shape
    test_x,test_y = test_features.shape

    x_train_reshaped = np.reshape(train_features,(train_x,1,train_y))
    x_test_reshaped = np.reshape(test_features,(test_x,1,test_y))

    lstm = lstm_model()
    lstm.fit(x_train_reshaped, y_train, batch_size=128, epochs=70)
    
    y_pred = lstm.predict(x_test_reshaped)
    
    # evaluate the model
    result = mean_squared_error(y_test.values,y_pred)
    print("MSE: {}".format(result))
    results.append(result)
    fold_cnt +=1

    tf.keras.backend.clear_session()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Fold 1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 300)            1282800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 1,376,305
Trainable params: 1,376,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/70
7/7 [==============================] - 22s 10ms/step - loss: 0.2258 - mae: 0.4036
Epoch 2/70
7/7 [==============================] - 0s 9ms/step - loss: 0.0506 - mae: 0.1797
Epoch 3/70
7/7 [==============================] - 0s 

NameError: ignored

In [ ]:
total_loss.append(results)
print("mean squared error is : {}".format(np.mean(np.asarray(results))))

In [ ]:
semantic_score = y_pred

In [ ]:
semantic_score

## prompt relevant score

In [ ]:
words = re.compile(r"\w+",re.I)
stopword = stopwords.words('english')
from nltk.stem.porter import *
stemmer = PorterStemmer()
import gensim
from gensim import corpora
from sklearn.metrics.pairwise import cosine_similarity as cs
from sklearn.metrics.pairwise import manhattan_distances as md
from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics import jaccard_similarity_score as jsc
from sklearn.neighbors import DistanceMetric
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_p = train.merge(prompt,on='essay_set',how='left')
train_p = train_p[['essay','normalized_score','prompt']]

In [ ]:
def tokenize(df):
    essay_tokenized = []
    prompt_tokenized = []

    for q in df.essay.tolist():
        essay_tokenized.append([stemmer.stem(i.lower()) for i in words.findall(q) if i not in stopword])

    for q in df.prompt.tolist():
        prompt_tokenized.append([stemmer.stem(i.lower()) for i in words.findall(q) if i not in stopword])

    df["essay_tokenized"] = essay_tokenized
    df["prompt_tokenized"] = prompt_tokenized

In [ ]:
tokenize(train_p)

In [ ]:
minkowski_dis = DistanceMetric.get_metric('minkowski')
mms_scale_man = MinMaxScaler()
mms_scale_euc = MinMaxScaler()
mms_scale_mink = MinMaxScaler()

In [ ]:
def get_similarity_values(q1_csc, q2_csc):
    cosine_sim = []
    manhattan_dis = []
    eucledian_dis = []
    jaccard_dis = []
    minkowsk_dis = []
    
    for i,j in zip(q1_csc, q2_csc):
        sim = cs(i,j)
        cosine_sim.append(sim[0][0])
        sim = md(i,j)
        manhattan_dis.append(sim[0][0])
        sim = ed(i,j)
        eucledian_dis.append(sim[0][0])
        i_ = i.toarray()
        j_ = j.toarray()
        try:
            sim = jsc(i_,j_)
            jaccard_dis.append(sim)
        except:
            jaccard_dis.append(0)
        sim = minkowski_dis.pairwise(i_,j_)
        minkowsk_dis.append(sim[0][0])
    return cosine_sim, manhattan_dis, eucledian_dis, jaccard_dis, minkowsk_dis    


## Handcrafted Features

In [ ]:
def get_correct_and_incorrect_spelling(df):
  """
  Function that measures lexical diversity which is the ratio of total words to unique words
  """
  # Load spaCy model
  nlp = spacy.load('en_core_web_sm')
  # load the enchant dictionary
  d = enchant.Dict("en_US")
  
  incorrect_spelling = []
  correct_spelling = []
  
  for essay in df['essay']:
    individual = []
    for word in essay.split():
      individual.append(d.check(word))                                                                         
    incorrect_spelling.append(individual.count(False))
    correct_spelling.append(individual.count(True))
  return incorrect_spelling, correct_spelling


def get_lexical_diversity(df):
    """
    Function that measures lexical diversity which is
    The ratio of total words to unique words
    """
    diversity = []
    for essay in df['essay']:
      diversity.append(round(len(tk.word_tokenize(essay)) / float(len(set(tk.word_tokenize(essay)))), 2))
    return diversity


def get_list_of_number_of_pos(df):
    """
    Function that parses the essay for each words POS
    Returns tuples containg for now, nouns, verbs, adverbs and adjectives
    """
    noun_cnt = []
    verb_cnt= []
    adv_cnt = []
    adj_cnt = []
    
    nlp = spacy.load('en_core_web_sm')
    for essay in df['essay']:
        parsed_essay = nlp(essay)
        token_pos = [token.pos_ for token in parsed_essay]
        noun_cnt.append(token_pos.count('NOUN'))
        verb_cnt.append(token_pos.count('VERB'))
        adv_cnt.append(token_pos.count('ADV'))
        adj_cnt.append(token_pos.count('ADJ'))   
    return noun_cnt, verb_cnt, adv_cnt,adj_cnt

In [ ]:
incorrect_spelling = get_correct_and_incorrect_spelling(train)[0]
correct_spelling = get_correct_and_incorrect_spelling(train)[1]

In [ ]:
lexical_diversity = get_lexical_diversity(train)

In [ ]:
noun_cnt = get_list_of_number_of_pos(train)[0]
verb_cnt = get_list_of_number_of_pos(train)[1]
adv_cnt = get_list_of_number_of_pos(train)[2]
adj_cnt = get_list_of_number_of_pos(train)[3]